In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_result = pd.read_csv("../input/english-premier-league-results/results.csv", encoding='cp949')
df_result

In [ ]:
df_result.info()

In [ ]:
df_result.isna().sum()

In [ ]:
df_result.drop(['DateTime','HTHG','HTAG','HTR','Referee', 'HS', 'AS','HST','AST', 'HC', 'AC', 'HF','AF', 'HY', 'AY', 'HR', 'AR']
              , axis=1, inplace=True)
df_result

In [ ]:
df_result.replace({'H' : 0, 'D' : 1, 'A' : 2}, inplace=True)
df_result

In [ ]:
# column명 추출
team_name = list(df_result["HomeTeam"].unique())

# DataFrame 생성
df_team = pd.DataFrame(columns=team_name, index=["G", "D"])

# 골득실 df_team에 널기
for i in team_name:
    df_team[i].G = sum(df_result["FTHG"][df_result["HomeTeam"]== i])
    df_team[i].G += sum(df_result["FTAG"][df_result["AwayTeam"] == i])
    df_team[i].D = sum(df_result["FTAG"][df_result["HomeTeam"] == i])
    df_team[i].G += sum(df_result["FTHG"][df_result["AwayTeam"] == i])
    
df_team

In [ ]:
# 골득점이 1000골 이상이 안되는 팀은 삭제
for i in team_name:
    team_G = df_team[i].loc['G'] < 1000
    if team_G:
        df_team.drop([i], axis=1, inplace=True)

df_team

In [ ]:
# 그래프를 그리그 위해 전치행렬로 바꾸기
df_team = df_team.T
df_team

In [ ]:
# 인덱스를 정수로 바꾸기
df_team = df_team.reset_index()
df_team.rename(columns = {'index' : 'team'}, inplace = True)
df_team_G = df_team.sort_values("G", ascending=False)

In [ ]:
# 골득실 그래프
figure, (ax1,ax2) = plt.subplots(nrows = 1, ncols =2 ) 
figure.set_size_inches(18,12)
sns.barplot(data=df_team_G, x="G", y="team", ax=ax1)
sns.barplot(data=df_team_G, x="D", y="team", ax=ax2)
ax1.set_xlabel("Goal", fontsize=30)
ax2.set_xlabel("Difference", fontsize=30)

In [ ]:
# 승률 그래프 (승리 경기 수) / (전체 경기 수)
cnt = 0

# 새로운 column인 win rate 생성
df_team.insert(3, "Winrate", 0)

# 경기 날짜는 2000년대 이후로 바꾼다.
i = 0
for i in range(0, 10802):
    season = (df_result["Season"].loc[i]).split("-")
    mm = int(season[0])
    if mm < 2000:
        df_result.drop(i, inplace=True)

for name in team_name:
    All_match = sum(df_result["HomeTeam"]==name) + sum(df_result["AwayTeam"]==name)
    Win_match = sum((df_result["HomeTeam"]==name) & (df_result["FTR"]==0))
    Win_match += sum((df_result["AwayTeam"]==name) & (df_result["FTR"]==2))
    if All_match == 0:
        continue
    df_team["Winrate"].loc[cnt] = round((Win_match/All_match) * 100, 1)
    cnt+=1
    
df_team

In [ ]:
# 승률 그래프

figure, (ax1,ax2) = plt.subplots(nrows = 1, ncols =2 ) 
figure.set_size_inches(18,12)
sns.barplot(data=df_team_G, x="G", y="team", ax=ax1)
sns.barplot(data=df_team.sort_values("Winrate", ascending=False), x="Winrate", y="team", ax=ax2)
ax1.set_xlabel("Goal", fontsize=30)
ax2.set_xlabel("Win rate", fontsize=30)

In [ ]:
df_result.drop(["Season", "FTHG", "FTAG"], axis=1, inplace=True)

In [ ]:
df_result

In [ ]:
df_en_result = pd.get_dummies(df_result, columns=["HomeTeam", "AwayTeam"])
df_en_result

In [ ]:
df_en_result.columns

In [ ]:
df_en_result.info()

In [ ]:
# Preprocessing
y = df_en_result.iloc[:, 2].values
del df_en_result["FTR"]
X = df_en_result.values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)